In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
from sklearn.metrics import *
from sklearn import model_selection
from sklearn import ensemble
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from h2o.automl import H2OAutoML
import h2o

In [2]:
import numpy as np
import pandas as pd

In [9]:
data_train = pd.read_csv('input/train.csv')
data_test = pd.read_csv('input/test.csv')

###  Data TEST LIMPIEZA

In [10]:
data_test

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,2.01,Ideal,H,SI1,61.9,57.0,8.14,8.05,5.01
1,1,0.49,Good,D,VS1,57.5,60.0,5.18,5.25,3.00
2,2,1.03,Premium,F,SI1,58.6,62.0,6.65,6.60,3.88
3,3,0.90,Very Good,E,SI1,63.0,56.0,6.11,6.15,3.86
4,4,0.59,Ideal,D,SI1,62.5,55.0,5.35,5.40,3.36
...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.41,Good,F,I1,63.8,57.0,4.72,4.69,3.00
13481,13481,2.05,Very Good,H,VS2,63.5,62.0,8.05,7.91,5.07
13482,13482,2.22,Premium,I,VS2,60.5,59.0,8.41,8.37,5.08
13483,13483,0.51,Ideal,E,SI2,58.3,62.0,5.20,5.19,3.04


In [6]:
data_test.describe()

,id,carat,depth,table,x,y,z
count,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000
mean,6742.000000,0.796130,61.758146,57.461528,5.730822,5.732280,3.537746
std,3892.928525,0.468802,1.419045,2.251915,1.111080,1.102966,0.688594
min,0.000000,0.200000,52.700000,49.000000,0.000000,0.000000,0.000000
25%,3371.000000,0.400000,61.000000,56.000000,4.720000,4.730000,2.920000
50%,6742.000000,0.700000,61.800000,57.000000,5.700000,5.710000,3.520000
75%,10113.000000,1.040000,62.500000,59.000000,6.530000,6.530000,4.030000
max,13484.000000,4.500000,71.300000,95.000000,10.230000,10.160000,6.720000


In [7]:
data_test.isnull().sum()

id         0
carat      0
cut        0
color      0
clarity    0
depth      0
table      0
x          0
y          0
z          0
dtype: int64

In [8]:
# Aqui podemos observar que cut, color y claroty son object

In [9]:
data_test.dtypes

id           int64
carat      float64
cut         object
color       object
clarity     object
depth      float64
table      float64
x          float64
y          float64
z          float64
dtype: object

In [ ]:
# Podemos pasar las columnas object a numericos, pero es mejor utilizar dummies.
''''
colors = {'F':1,
          'D':2,
          'E':3,
          'J':4,
          'H':5,
          'I':6,
          'G':7}
cut = {"Premium":5,
          "Ideal":4,
          "Very Good":3,
          "Good":2,
          "Fair":1}
clarity = {'VS1':1,
           'SI1':2,
           'VVS2':3,
           'SI2':4,
           'VS2':5,
           'IF':6,
           'VVS1':7,
           'I1':8}
'''

#### Con dummies pasamos las columnas a numericas

In [11]:
df_dummies = pd.get_dummies(data_test)

df_dummies.columns

Index(['id', 'carat', 'depth', 'table', 'x', 'y', 'z', 'cut_Fair', 'cut_Good',
       'cut_Ideal', 'cut_Premium', 'cut_Very Good', 'color_D', 'color_E',
       'color_F', 'color_G', 'color_H', 'color_I', 'color_J', 'clarity_I1',
       'clarity_IF', 'clarity_SI1', 'clarity_SI2', 'clarity_VS1',
       'clarity_VS2', 'clarity_VVS1', 'clarity_VVS2'],
      dtype='object')

In [12]:
df_dummies.drop(['id'],axis=1,inplace=True)

In [13]:
df_dummies

,carat,depth,table,x,y,z,cut_Fair,cut_Good,cut_Ideal,cut_Premium,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,2.01,61.9,57.0,8.14,8.05,5.01,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1,0.49,57.5,60.0,5.18,5.25,3.00,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2,1.03,58.6,62.0,6.65,6.60,3.88,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,0.90,63.0,56.0,6.11,6.15,3.86,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0.59,62.5,55.0,5.35,5.40,3.36,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0.41,63.8,57.0,4.72,4.69,3.00,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
13481,2.05,63.5,62.0,8.05,7.91,5.07,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
13482,2.22,60.5,59.0,8.41,8.37,5.08,0,0,0,1,...,1,0,0,0,0,0,0,1,0,0
13483,0.51,58.3,62.0,5.20,5.19,3.04,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
df_dummies.corr()

In [ ]:
columns = ['carat', 'cut', 'color', 'clarity', 'depth']

In [ ]:
data_test

In [ ]:
data_test.drop(["depth","table", "id"], axis = 1, inplace = True) 

In [ ]:
data_test

In [ ]:
# lo guardamos

In [15]:
df_dummies.to_csv('output/limpio_test.csv')

### Exportamos el model de X_train Gradient Boosting Regression

In [14]:
gradient_boosting_model = pickle.load(open("gradient_boosting_model", 'rb'))

In [20]:
gradient_boosting_model.predict(df_dummies)

array([9.66783919, 7.31363316, 8.49802339, ..., 9.7106148 , 7.05853469,
       7.17028684])

In [21]:
price = gradient_boosting_model.predict(df_dummies)

In [22]:
df_dummies

,carat,depth,table,x,y,z,cut_Fair,cut_Good,cut_Ideal,cut_Premium,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,2.01,61.9,57.0,8.14,8.05,5.01,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1,0.49,57.5,60.0,5.18,5.25,3.00,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2,1.03,58.6,62.0,6.65,6.60,3.88,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,0.90,63.0,56.0,6.11,6.15,3.86,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0.59,62.5,55.0,5.35,5.40,3.36,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0.41,63.8,57.0,4.72,4.69,3.00,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
13481,2.05,63.5,62.0,8.05,7.91,5.07,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
13482,2.22,60.5,59.0,8.41,8.37,5.08,0,0,0,1,...,1,0,0,0,0,0,0,1,0,0
13483,0.51,58.3,62.0,5.20,5.19,3.04,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [23]:
df_dummies["price"] = price

In [25]:
data_submission = df_dummies

In [27]:
data_submission = data_submission.drop(columns = ['carat', 'depth', 'table', 'x', 'y', 'z', 'cut_Fair', 'cut_Good',
       'cut_Ideal', 'cut_Premium', 'cut_Very Good', 'color_D', 'color_E',
       'color_F', 'color_G', 'color_H', 'color_I', 'color_J', 'clarity_I1',
       'clarity_IF', 'clarity_SI1', 'clarity_SI2', 'clarity_VS1',
       'clarity_VS2', 'clarity_VVS1', 'clarity_VVS2'])

In [40]:
data_submission.reset_index(inplace = True)

In [41]:
data_submission.rename(columns={"index": "id"}, inplace = True)

In [42]:
data_submission

,id,price
0,0,9.667839
1,1,7.313633
2,2,8.498023
3,3,8.356842
4,4,7.495482
...,...,...
13480,13480,6.441047
13481,13481,9.623936
13482,13482,9.710615
13483,13483,7.058535


In [43]:
data_submission.to_csv('submission.csv', index = False)

In [ ]:
##  Model: Gradient Boosting Regression

In [ ]:
gradient_boosting_model = pickle.load(open("gradient_boosting_model", 'rb'))

In [ ]:
### Exportamos el modelo H20

In [15]:
path = "./h20_model"

In [ ]:
imported_model = h2o.import_mojo(path)

In [20]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,27 mins 47 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.2
H2O_cluster_version_age:,9 days
H2O_cluster_name:,H2O_from_python_Dario_kcqygz
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.666 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [21]:
h2o_frame = h2o.H2OFrame(df_dummies)

Parse progress: |█████████████████████████████████████████████████████████| 100%
